In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/תואר שני/פרויקט גמר/Model Training

/content/drive/MyDrive/תואר שני/פרויקט גמר/Model Training


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.6 MB/s eta 0:00:00


In [ ]:
import numpy as np
import keras
import pandas as pd
pd.options.mode.chained_assignment = None 
import tensorflow as tf
import transformers
from transformers import logging
logging.set_verbosity_error()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
tokenizer = transformers.BertTokenizerFast.from_pretrained('onlplab/alephbert-base')
bert_base = transformers.TFBertModel.from_pretrained('onlplab/alephbert-base')

Downloading:   0%|          | 0.00/288 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/545k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666M [00:00<?, ?B/s]

In [27]:
model = tf.keras.models.load_model('alephbert_finetuned_model_v2',custom_objects={'TFBertModel':bert_base},compile = False)

In [28]:
max_length = 128  # Maximum length of input sentence to the model.
batch_size = 32
# epochs = 2

# Labels in our dataset.
labels = ["negative", "positive"]

In [29]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use onlplab/alephbert-base pretrained model.
        
        self.tokenizer = transformers.BertTokenizerFast.from_pretrained('onlplab/alephbert-base')
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)

In [30]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )
    proba = model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    pred = labels[idx]
    return pred, proba

In [31]:
# sentences are positive
sentence1 = "מה אתה יודע לעשות?"
sentence2 = "מה הפעולות שאתה יודע לעשות?"
check_similarity(sentence1, sentence2)

1/1 [==============================] - 6s 6s/step


('positive', array([0.00258928, 0.9974107 ], dtype=float32))

In [32]:
sentence1 = "מה אתה יודע לעשות?"
sentence2 = "מה התחביבים שלך?"
check_similarity(sentence1, sentence2)

1/1 [==============================] - 0s 65ms/step


('negative', array([0.9694614 , 0.03053858], dtype=float32))

In [33]:
sentence1 = "החה חעח עצ חלעיצ יעלג"
sentence2 = "פעולת ריקוד"
check_similarity(sentence1, sentence2)

1/1 [==============================] - 0s 61ms/step


('positive', array([0.01247048, 0.9875295 ], dtype=float32))

In [34]:
default_list = pd.read_csv('default_sentence_list_utf8.csv')
default_list.head()

,default sentence list
0,מה הפעולות שאתה יודע לעשות?
1,איך אתה מרגיש?
2,האם עשית משהו מעניין היום?
3,אני מרגיש טוב ורוצה לדבר על זה
4,מה קרה לך?


# Comparing user sentence with the reference sentences

In [35]:
def reference_similarity_all(user_sentence,threshold,default_list):
  all_score = []
  most_similar_sentence = ""
  similarity_type = 0
  
  # comparing reference sentence with default list
  for idx in range(len(default_list)):
    reference_sentence = default_list[idx]
    all_score.append(check_similarity(user_sentence, reference_sentence)[1][1])

  # checking the maximum score and return sentence for maximum score even if belos threshold
  max_val = max(all_score)
  item_idx = np.where(all_score==max_val)
  item_idx = item_idx[0][0]
  most_similar_sentence = default_list[item_idx]

  
  if max_val >= threshold:
    similarity_type = 1   # to check if the score is above the threshold


  return (most_similar_sentence, similarity_type, max_val)

In [36]:
def reference_similarity_one_by_one(user_sentence,threshold,default_list):
  all_score = []
  most_similar_sentence = ""
  similarity_type = 0
  
  # comparing reference sentence with default list
  for idx in range(len(default_list)):
    reference_sentence = default_list[idx]
    current_score = check_similarity(user_sentence, reference_sentence)[1][1]
    all_score.append(current_score)
    if current_score >= threshold:
      similarity_type = 1
      return (reference_sentence, similarity_type, current_score)

  # checking the maximum score and return sentence for maximum score even if belos threshold
  max_val = max(all_score)
  item_idx = np.where(all_score==max_val)
  item_idx = item_idx[0][0]
  most_similar_sentence = default_list[item_idx]

  return (most_similar_sentence, similarity_type, max_val)

In [37]:
user_sentence = "מה אתה יודע לעשות?"
threshold = 0.8
default_lst = default_list['default sentence list']
most_sim_sentence = reference_similarity_all(user_sentence, threshold, default_lst)
print(most_sim_sentence)
most_sim_sentence_new = reference_similarity_one_by_one(user_sentence, threshold, default_lst)
print(most_sim_sentence_new)

1/1 [==============================] - 0s 55ms/step
('מה הפעולות שאתה יודע לעשות?', 1, 0.9974107)
1/1 [==============================] - 0s 57ms/step
('מה הפעולות שאתה יודע לעשות?', 1, 0.9974107)


In [38]:
user_sentence = "אתה זוכר משהו?"
threshold = 0.8
default_lst = default_list['default sentence list']
most_sim_sentence = reference_similarity_all(user_sentence, threshold, default_lst)
print(most_sim_sentence)
most_sim_sentence_new = reference_similarity_one_by_one(user_sentence, threshold, default_lst)
print(most_sim_sentence_new)

1/1 [==============================] - 0s 62ms/step
('מה זכרונות הילדות שלך?', 1, 0.9952058)
1/1 [==============================] - 0s 62ms/step
('תספר לי על המשפחה שלך', 1, 0.8071699)


In [39]:
user_sentence = "בוא נזיז את הרגליים"
threshold = 0.8
default_lst = default_list['default sentence list']
most_sim_sentence = reference_similarity_all(user_sentence, threshold, default_lst)
print(most_sim_sentence)
most_sim_sentence_new = reference_similarity_one_by_one(user_sentence, threshold, default_lst)
print(most_sim_sentence_new)

1/1 [==============================] - 0s 58ms/step
('פעולת ריקוד', 0, 0.27115375)
1/1 [==============================] - 0s 59ms/step
('פעולת ריקוד', 0, 0.27115375)


In [40]:
user_sentence = "חתולים וכלבים"
threshold = 0.8
default_lst = default_list['default sentence list']
most_sim_sentence = reference_similarity_all(user_sentence, threshold, default_lst)
print(most_sim_sentence)
most_sim_sentence_new = reference_similarity_one_by_one(user_sentence, threshold, default_lst)
print(most_sim_sentence_new)

1/1 [==============================] - 0s 61ms/step
('תספר לי על המשפחה שלך', 0, 0.5137551)
1/1 [==============================] - 0s 68ms/step
('תספר לי על המשפחה שלך', 0, 0.5137551)


In [41]:
user_sentence = "יכחלגד ךיכך ךלחכדלג"
threshold = 0.8
default_lst = default_list['default sentence list']
most_sim_sentence = reference_similarity_all(user_sentence, threshold, default_lst)
print(most_sim_sentence)
most_sim_sentence_new = reference_similarity_one_by_one(user_sentence, threshold, default_lst)
print(most_sim_sentence_new)

1/1 [==============================] - 0s 111ms/step
('פעולת ריקוד', 1, 0.9961287)
1/1 [==============================] - 0s 58ms/step
('מה התחביבים שאתה אוהב לעשות?', 1, 0.9322051)


In [24]:
user_sentence = "החה חעח עצ חלעיצ יעלג"
threshold = 0.8
default_lst = default_list['default sentence list']
most_sim_sentence = reference_similarity_all(user_sentence, threshold, default_lst)
print(most_sim_sentence)
most_sim_sentence_new = reference_similarity_one_by_one(user_sentence, threshold, default_lst)
print(most_sim_sentence_new)

1/1 [==============================] - 0s 56ms/step
('פעולת ריקוד', 1, 0.9875295)
1/1 [==============================] - 0s 58ms/step
('במה עבדת?', 1, 0.84544617)
